# Exploring Hacker News posts

In this project, we'll compare two different types of posts from Hacker News. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?  
Do posts created at a certain time receive more comments on average?  
It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction
First, we'll read in the data and remove the headers.

In [18]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[0:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Removing Headers from a List of Lists

In [19]:
headers = hn[0]
hn = hn[1:]

print(headers)
print('\n')
print(hn[0:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]



We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

## Extracting Ask HN and Show HN Posts  
First, we'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [20]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if (title.lower()).startswith('ask hn') == True:
        ask_posts.append(row)
    elif (title.lower()).startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))
print()
print(ask_posts[0:5])

1744
1162
17194

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


## Calculating the Average Number of Comments for Ask HN and Show HN Posts  

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [21]:
total_ask_comments = 0

for row in ask_posts:
    comment = int(row[4])
    total_ask_comments += comment

avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [22]:
total_show_comments = 0

for row in show_posts:
    comment = int(row[4])
    total_show_comments += comment

avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.


## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [29]:
import datetime as dt

result_list = []

for row in ask_posts:
    created = row[6]
    comment = int(row[4])
    
    result_list.append([created, comment])
    

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    time_hour = dt.datetime.strftime(time, "%H")
    comment = int(row[1])
   

    if time_hour in counts_by_hour:
        comments_by_hour[time_hour] += comment
        counts_by_hour[time_hour] += 1
    else:
        comments_by_hour[time_hour] = comment
        counts_by_hour[time_hour] = 1
    

print(counts_by_hour) 
print()
print(comments_by_hour)

{'18': 109, '11': 58, '10': 59, '17': 100, '08': 48, '09': 45, '15': 116, '22': 71, '07': 34, '04': 47, '16': 108, '06': 44, '20': 80, '01': 60, '02': 58, '19': 110, '14': 107, '23': 68, '00': 55, '12': 73, '05': 46, '13': 85, '21': 109, '03': 54}

{'18': 1439, '11': 641, '10': 793, '17': 1146, '08': 492, '09': 251, '15': 4477, '22': 479, '07': 267, '04': 337, '16': 1814, '06': 397, '20': 1722, '01': 683, '02': 1381, '19': 1188, '14': 1416, '23': 543, '00': 447, '12': 687, '05': 464, '13': 1253, '21': 1745, '03': 421}


### Calculating the Average Number of Comments for Ask HN Posts by Hour

In [30]:
avg_by_hour = []

for hour in comments_by_hour:
    avg_by_hour.append([hour, (comments_by_hour[hour]/counts_by_hour[hour])])
    
    
avg_by_hour

[['18', 13.20183486238532],
 ['11', 11.051724137931034],
 ['10', 13.440677966101696],
 ['17', 11.46],
 ['08', 10.25],
 ['09', 5.5777777777777775],
 ['15', 38.5948275862069],
 ['22', 6.746478873239437],
 ['07', 7.852941176470588],
 ['04', 7.170212765957447],
 ['16', 16.796296296296298],
 ['06', 9.022727272727273],
 ['20', 21.525],
 ['01', 11.383333333333333],
 ['02', 23.810344827586206],
 ['19', 10.8],
 ['14', 13.233644859813085],
 ['23', 7.985294117647059],
 ['00', 8.127272727272727],
 ['12', 9.41095890410959],
 ['05', 10.08695652173913],
 ['13', 14.741176470588234],
 ['21', 16.009174311926607],
 ['03', 7.796296296296297]]

## Sorting and Printing Values from a List of Lists

Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [33]:
swap_avg_by_hour = []

for item in avg_by_hour:
    swap_avg_by_hour.append([item[1],item[0]])



sorted_swap = sorted(swap_avg_by_hour, reverse = True)

sorted_swap


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [34]:
print("Top 5 Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )

Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


### Conclusion
The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post.